In [29]:
import pandas as pd
import ast
import json

In [30]:
df = pd.read_csv('/Users/muhammadluay/Desktop/Zindi/IDRISI-main/scripts/clean/check_detection.csv')

In [31]:
df

,user_id,tweet_id,text,humAID_class,entities_bert_large,entities_bert_base,locations_stanza,entities_suweilah,location_mentions,false_positives_re,pred_re,labels,all_labels_re,extracted_locations_bert_large,extracted_locations_bert_base_labels,entities_suweilah_labels
0,2.458177e+09,1.040000e+18,"Three days after #Florence made landfall, the ...",rescue_volunteering_or_donation_effort,"[{'entity': 'I-MISC', 'score': 0.7631275, 'ind...","[{'entity': 'B-LOC', 'score': 0.965381, 'index...","['Carolinas', 'Connecticut']","[{'entity': 'U-STAT', 'score': 0.5439643, 'ind...","[{'text': 'Connecticut', 'type': 'State', 'sta...","['american red cross', 'nd', 'fl', 'li', 'or',...","['connecticut', 'carolinas']",['connecticut'],"['the carolinas', 'ne', 'america', 'carolinas'...","['Carolinas', 'Connecticut']","['Florence', 'Carolinas', 'Connecticut']","['Carolina', 'Connecticut']"
1,1.010000e+18,1.040000e+18,"Florence, now a Category 2 hurricane, is forec...",caution_and_advice,"[{'entity': 'I-PER', 'score': 0.6463937, 'inde...","[{'entity': 'B-LOC', 'score': 0.9984102, 'inde...",['Florence'],"[{'entity': 'U-CITY', 'score': 0.4929503, 'ind...","[{'text': 'Florence', 'type': 'City/town', 'st...","['nd', 'fl', 'or', 'ma', 'ne', 'nc', 'ca', 'un...",['florence'],['florence'],"['ma', 'ne', 'nc', 'florence', 'ca', 'in', 'fl...",[],['Florence'],['Florence']
2,3.750983e+07,1.040000e+18,RT @annamoor1995: Rescue the #Pigs #Trapped Du...,rescue_volunteering_or_donation_effort,"[{'entity': 'I-ORG', 'score': 0.7256747, 'inde...","[{'entity': 'I-LOC', 'score': 0.87438077, 'ind...",[],[],[],"['fl', 'sc', 'or', 'ne', 'nc', 'ca', 'hurrican...",[],[],"['ne', 'sc', 'nc', 'ca', 'florence', 'in', 'fl...",[],['Florence'],[]
3,9.750000e+17,1.040000e+18,"In Winnabow, North Carolina some residents are...",requests_or_urgent_needs,"[{'entity': 'I-LOC', 'score': 0.9962436, 'inde...","[{'entity': 'B-LOC', 'score': 0.9920365, 'inde...","['Winnabow', 'North Carolina', 'Wilmington']","[{'entity': 'U-CITY', 'score': 0.63645846, 'in...","[{'text': 'Winnabow', 'type': 'County', 'start...","['nd', 'fl', 'li', 'or', 'up', 'ne', 'highway ...","['winnabow', 'wilmington', 'florence', 'north ...","['winnabow', 'north carolina', 'wilmington', '...","['ne', 'winnabow', 'or', 'florence', 'carolina...","['Winnabow', 'NorthCarolina', 'Wilmington', 'H...","['Winnabow', 'NorthCarolina', 'Wilmington', 'F...","['Win', 'NorthCarolina', 'Wilmington']"
4,1.436126e+07,1.040000e+18,#Flood Safety: Avoid driving through pools of ...,caution_and_advice,[],"[{'entity': 'B-LOC', 'score': 0.87995416, 'ind...",['Florence'],[],[],"['nd', 'fl', 'or', 'nc', 'sa', 'wa', 'un', 'mt...",[],[],"['sa', 'nc', 'mt', 'florence', 'in', 'fl', 'wa...",[],['Florence'],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443,8.690000e+17,8.730000e+17,RT @Iyoshan: respect you Skipper @Angelo69Math...,rescue_volunteering_or_donation_effort,"[{'entity': 'I-ORG', 'score': 0.57353944, 'ind...","[{'entity': 'B-ORG', 'score': 0.36107796, 'ind...",[],[],[],"['nd', 'fl', 'or', 'ma', 'ca', 'in']",[],[],"['in', 'ma', 'fl', 'nd', 'ca', 'or']",['Iyo'],[],[]
16444,8.610000e+17,8.700000e+17,·º∞F Please donate funds for #SriLanka ·ºü1·ºü...,rescue_volunteering_or_donation_effort,"[{'entity': 'I-MISC', 'score': 0.9017203, 'ind...","[{'entity': 'B-LOC', 'score': 0.71410745, 'ind...",[],"[{'entity': 'U-CTRY', 'score': 0.98769623, 'in...","[{'text': 'SriLanka', 'type': 'Country', 'star...","['srilank', 'lka', 'nd', 'fl', 'or', 'ne', 'nc...",['srilanka'],['srilanka'],"['ne', 'nc', 'or', 'in', 'srilanka', 'redcross...",[],"['Sri', 'Sri']",['SriLka']
16445,8.690000e+17,8.700000e+17,RT @AmanthaP: APPEAL @SLRedCross running low o...,requests_or_urgent_needs,"[{'entity': 'I-LOC', 'score': 0.58190244, 'ind...","[{'entity': 'B-LOC', 'score': 0.6180288, 'inde...","['Gampaha', 'Matara']","[{'entity': 'U-CITY', 'score': 0.9448759, 'ind...","[{'text': 'Gampaha', 'type': 'City/town', 'sta...","['fl', 'ma', 'ni', 'cal', 'dc', 'g

In [32]:
df['entities_suweilah'][0]

"[{'entity': 'U-STAT', 'score': 0.5439643, 'index': 16, 'word': 'Carolina', 'start': 75, 'end': 83}, {'entity': 'U-STAT', 'score': 0.90131927, 'index': 27, 'word': 'Connecticut', 'start': 134, 'end': 145}]"

In [36]:
def merge_words(entities_str):
    entities = json.loads(entities_str.replace("'", '"'))
    processed_entities = []
    word_cache = ''
    entity_cache = None
    
    for entity in entities:
        word = entity['word']

        # Check if the current entity should be merged with the cached entity
        should_merge = (
            entity_cache is not None and 
            entity['start'] <= entity_cache['end'] + 2
        )
        
        # Including special characters into the previous entity
        if word in ['.', '/'] and entity_cache is not None:
            entity_cache['word'] += word
            entity_cache['end'] = entity['end']
            continue
        
        if should_merge or (word.startswith('##') and entity_cache is not None):
            word = word[2:] if word.startswith('##') else ' ' + word
            word_cache += word
            entity_cache['end'] = entity['end']
            entity_cache['word'] += word
        else:
            if entity_cache is not None:
                processed_entities.append(entity_cache)
            
            word_cache = word
            entity_cache = entity.copy()
    
    if entity_cache is not None:
        processed_entities.append(entity_cache)
    
    return processed_entities

# Applying the function to process entities and create the new 'processed_suweilah_entities' column
df['processed_suweilah_entities'] = df['entities_suweilah'].apply(merge_words)

In [37]:
df['processed_suweilah_entities'][521]


[{'entity': 'B-ST',
  'score': 0.5937734,
  'index': 6,
  'word': '3165 Olentangy River Rd',
  'start': 23,
  'end': 46}]

In [38]:
df['location_mentions'][16447]

"[{'text': 'SriLanka', 'type': 'Country', 'start_offset': 16, 'end_offset': 24}]"

In [39]:
df['processed_suweilah_labels'] = df['processed_suweilah_entities'].apply(lambda entities: [entity['word'] for entity in entities])


In [40]:
df[['processed_suweilah_labels','labels', 'entities_suweilah_labels']].head(50)

,processed_suweilah_labels,labels,entities_suweilah_labels
0,"[Carolina, Connecticut]",['connecticut'],"['Carolina', 'Connecticut']"
1,[Florence],['florence'],['Florence']
2,[],[],[]
3,"[Win, North Carolina, Wilmington]","['winnabow', 'north carolina', 'wilmington', '...","['Win', 'NorthCarolina', 'Wilmington']"
4,[],[],[]
5,[],[],[]
6,[],[],[]
7,[],[],[]
8,[Seven Springs],['seven springs'],['SevenSprings']
9,[],[],[]


In [41]:
malformed_labels = []  # This list will store dictionaries of malformed and correct labels

for index, row in df.iterrows():
    suweilah_entities = row['processed_suweilah_entities']
    location_mentions = ast.literal_eval(row['location_mentions'])
    
    for suweilah in suweilah_entities:
        suweilah_start = suweilah['start']
        suweilah_end = suweilah['end']
        suweilah_word = suweilah['word']
        
        for location in location_mentions:
            location_start = location['start_offset']
            location_end = location['end_offset']
            location_text = location['text']
            
            if suweilah_start == location_start:
                if suweilah_end < location_end:
                    malformed_labels.append({
                        "malformed_label": suweilah_word,
                        "correct_label": location_text
                    })
                elif suweilah_end > location_end:
                    next_loc_index = location_mentions.index(location) + 1
                    if next_loc_index < len(location_mentions) and \
                        location_mentions[next_loc_index]['start_offset'] == location_end + 1:
                        correct_label = location_text + ' ' + location_mentions[next_loc_index]['text']
                        malformed_labels.append({
                            "malformed_label": suweilah_word,
                            "correct_label": correct_label
                        })

In [42]:
malformed_suweilah = pd.DataFrame(malformed_labels, columns=["malformed_label", "correct_label"])


In [43]:
malformed_suweilah.drop_duplicates()

,malformed_label,correct_label
0,Win,Winnabow
1,Carolina,Carolinas
2,R,RICHMOND
3,NorthCaro,NorthCarolina
4,Virginia,Virginias
...,...,...
3172,Math,Mathugama
3175,Cho,Choluteca
3178,G,Galle
3179,Bad,Baddegama


In [44]:
malformed_suweilah.to_csv('malformed_suweilah.csv')